In [ ]:
import os
import nibabel as nib
import numpy as np
import pandas as pd

# -----------------------------
# CONFIGURACIÓN
# -----------------------------
NNUNET_DIR = "/media/guest/PORT-DISK/Practicas/MicroBleeds/nnUNet_raw_data/TaskVALDO"
imagesTr_dir = os.path.join(NNUNET_DIR, "imagesTr")
labelsTr_dir = os.path.join(NNUNET_DIR, "labelsTr")

# Definición de modalidades
MODALITIES = {
    "T2*": "_0000",
    "T1":  "_0001",
    "T2":  "_0002",
    "Mask": ""  # La máscara está en otra carpeta y no tiene sufijo numérico
}

def get_nifti_metadata(path, modality_name, sub_id):
    """Extrae metadatos del header sin cargar toda la imagen (más rápido)."""
    if not os.path.exists(path):
        return None

    try:
        nii = nib.load(path)
        header = nii.header
        
        # 1. Dimensiones (X, Y, Z)
        # Z suele ser el número de cortes en imágenes 3D médicas
        dims = nii.shape 
        
        # 2. Tamaño de Voxel (Spacing en mm)
        zooms = header.get_zooms()
        
        # 3. Orientación (RAS, LPS, etc.)
        # Convierte la matriz afín a códigos de letras (ej. ('R', 'A', 'S'))
        orientation = nib.aff2axcodes(nii.affine)
        
        # 4. Tipo de dato
        dtype = header.get_data_dtype()

        return {
            "Subject": sub_id,
            "Modality": modality_name,
            "Dim_X": dims[0],
            "Dim_Y": dims[1],
            "Dim_Z (Slices)": dims[2] if len(dims) > 2 else 1,
            "Voxel_X (mm)": round(zooms[0], 4),
            "Voxel_Y (mm)": round(zooms[1], 4),
            "Voxel_Z (mm)": round(zooms[2], 4),
            "Orientation": "".join(orientation),
            "DType": dtype
        }
    except Exception as e:
        print(f"Error leyendo {path}: {e}")
        return None

# -----------------------------
# EJECUCIÓN
# -----------------------------
# 1. Obtener lista de sujetos únicos
files = os.listdir(imagesTr_dir)
# Filtramos solo los que terminen en _0000.nii.gz para sacar el ID base
sub_ids = sorted([f.split("_")[0] for f in files if f.endswith("_0000.nii.gz")])

print(f"Detectados {len(sub_ids)} sujetos. Iniciando análisis...\n")

data_list = []

for sub_id in sub_ids:
    # Iterar por cada modalidad (T2*, T1, T2, Mask)
    for mod_name, suffix in MODALITIES.items():
        
        # Determinar ruta dependiendo si es imagen o label
        if mod_name == "Mask":
            filename = f"{sub_id}.nii.gz"
            path = os.path.join(labelsTr_dir, filename)
        else:
            filename = f"{sub_id}{suffix}.nii.gz"
            path = os.path.join(imagesTr_dir, filename)
            
        # Extraer datos
        meta = get_nifti_metadata(path, mod_name, sub_id)
        if meta:
            data_list.append(meta)
        else:
            # Registro de archivo faltante (opcional)
            print(f"Warning: Falta {mod_name} para sujeto {sub_id}")

# 2. Crear DataFrame de Pandas
df = pd.DataFrame(data_list)

      
# Agrupamos por modalidad para ver si todos tienen la misma resolución
# Esto es vital: nnU-Net prefiere que todos tengan mismo espaciado, 
# si no, hará resampling.

group_cols = ["Dim_X", "Dim_Y", "Dim_Z (Slices)", "Voxel_X (mm)", "Voxel_Y (mm)", "Voxel_Z (mm)"]

for mod in MODALITIES.keys():
    print(f"\n--- Análisis de Modalidad: {mod} ---")
    df_mod = df[df["Modality"] == mod]
    
    # Contamos cuántas combinaciones únicas de dimensiones/voxels existen
    unique_configs = df_mod[group_cols].value_counts()
    
    print(unique_configs)
    
    # Chequeo de Orientación
    orientations = df_mod["Orientation"].unique()
    if len(orientations) > 1:
        print(f"⚠️ ¡ALERTA! Orientaciones mixtas detectadas: {orientations}")
    else:
        print(f"Orientación consistente: {orientations[0]}")

# 3. Exportar a CSV (opcional, muy útil para abrir en Excel)
output_csv = "/media/guest/PORT-DISK/Practicas/MicroBleeds/analisis_resolucion_dataset.csv"

df.to_csv(output_csv, index=False)
print(f"\nAnálisis completo guardado en: {output_csv}")

# 4. Mostrar primeras filas del análisis:
print("\nPrimeras filas del análisis:")
print(df.head(10))

Detectados 72 sujetos. Iniciando análisis...


--- Análisis de Modalidad: T2* ---
Dim_X  Dim_Y  Dim_Z (Slices)  Voxel_X (mm)  Voxel_Y (mm)  Voxel_Z (mm)
512    512    192             0.4883        0.4883        0.8000          34
              35              0.4492        0.4492        4.0000          11
256    256    39              1.0000        1.0000        3.0000           7
                                                          3.0001           4
                                                          4.0001           3
                                                          4.0000           2
              36              1.0000        1.0000        3.9996           1
                                                          4.0004           1
              39              1.0000        1.0000        2.9995           1
                                                          3.9994           1
                                                          3.0004           1
